In [4]:
import json
import pathlib
import gzip
import ast
import clip
import pickle
from tqdm.notebook import tqdm
import sys
sys.path.append("/home/ubuntu/MLA/")
from habitat_extensions import maps
import numpy as np
def get_nearest_node(graph, current_position) -> str:
    """Determine the closest MP3D node to the agent's start position as given
    by a [x,z] position vector.
    Returns:
        node ID
    """
    nearest = None
    dist = float("inf")
    current_position = np.take(current_position, (0, 2))
    for node in graph:
        node_pos = graph.nodes[node]["position"]
        node_pos = np.take(node_pos, (0, 2))
        cur_dist = np.linalg.norm(
            np.array(node_pos) - np.array(current_position), ord=2
        )
        if cur_dist < dist:
            dist = cur_dist
            nearest = node
    return nearest, dist
def calc_dist(pos1, pos2):
    pos1 = np.take(pos1, (0, 2))
    pos2 = np.take(pos2, (0, 2))
    cur_dist = np.linalg.norm(
        np.array(pos1) - np.array(pos2), ord=2
    )
    return cur_dist



In [6]:
def generate_subpath(split):
    data_folder =  pathlib.Path("..","data","datasets","FGR2R")
    data_path = data_folder /  pathlib.Path("FGR2R_{}.json".format(split))
    with open(data_path, "r") as f:
        data = json.loads(f.read())

    sub_folder = pathlib.Path("..","data","datasets","R2R_VLNCE_FGSub")
    sub_path = sub_folder /  pathlib.Path("{}".format(split),"{}_sub.json.gz".format(split))
    with gzip.open(sub_path, "r") as f:
        sub_data = json.loads(f.read())
    if split!="test":
        gt_path = sub_folder /  pathlib.Path("{}".format(split),"{}_gt.json.gz".format(split))
        with gzip.open(gt_path, "r") as f:
            sub_gt = json.loads(f.read())

    graph_path = pathlib.Path("..","data","connectivity_graphs.pkl")
    with open(graph_path, "rb") as f:
        conn_graphs = pickle.load(f)

    # 使用trajectory_id重新组织episodes
    # 每个trajectory_id对应的第4条路经删去，并且去掉超长度限制215的路径
    sub_data_byid = {}
    trajectory_ids = set([v["trajectory_id"] for v in sub_data["episodes"]])
    # if split=="train":
    #     trajectory_ids.remove(4373)
    for i in trajectory_ids:
        eps = []
        for v in sub_data["episodes"]:
            if v["trajectory_id"] == i and len(eps)<3:
                eps.append(v)
        sub_data_byid[i] = eps
    lens = []
    for i in tqdm(trajectory_ids):
        origin = None
        for v in data:
            if v["path_id"]==i:
                origin = v
        # 替换sub_instructions
        origin_sub = [[" ".join(v) for v in a] for a in ast.literal_eval(origin["new_instructions"])]
        origin_tokens = [[clip.tokenize(v, truncate=True).squeeze(0).tolist() for v in a] for a in origin_sub]
        sub_num = 11
        zero_sub = [0]*77
        for j in range(3):
            sub_data_byid[i][j]["sub_instruction"] = origin_sub[j]
            tmp = origin_tokens[j]
            if len(tmp)>sub_num:
                tmp = tmp[0:sub_num]
            tmp.extend([zero_sub]*(sub_num-len(tmp)))
            sub_data_byid[i][j]["sub_instruction_tokens"] = tmp
        # 计算action_to_sub
        pad_len = 215
        if split!="test":
            origin_chunk = origin["chunk_view"]
            origin_path = origin["path"]
            for j in range(3):
                # 取出episode的信息
                ep = sub_data_byid[i][j]
                scene_id = ep["scene_id"].split("/")[1]
                graph = conn_graphs[scene_id]
                ep_gt = sub_gt[str(ep["episode_id"])]
                ep_path = ep_gt["locations"]
                ep_actions = ep_gt["actions"]
                # 计算对应每个action的坐标位置
                ep_path_full = []
                ep_path_idx = 0
                step_idx = 0
                while step_idx<len(ep_actions):
                    ep_path_full.append(ep_path[ep_path_idx])
                    if ep_actions[step_idx]==1:
                        ep_path_idx+=1
                    step_idx+=1
                
                chunk = origin_chunk[j]
                keynodes = []
                for c in chunk:
                    keynodes.append(origin_path[c[1]-1])
                sub_idx = 0
                step_idx = 0
                action_to_sub = np.zeros(len(ep_actions))
                # 每行动一步，判断是否到达当前子指令对应的终点
                avoid_multi = False
                while step_idx<len(ep_actions) or sub_idx<len(keynodes):
                    if step_idx==len(ep_actions):
                        action_to_sub[step_idx-1] = sub_idx
                        sub_idx += 1
                        continue
                    if sub_idx==len(keynodes):
                        action_to_sub[step_idx] = sub_idx-1
                        step_idx += 1
                        continue
                    action_to_sub[step_idx] = sub_idx
                    dist = calc_dist(
                        graph.nodes[keynodes[sub_idx]]["position"], ep_path_full[step_idx]
                    )
                    if dist<=0.6 and not avoid_multi:
                        sub_idx += 1
                        avoid_multi = True
                    elif dist>0.6:
                        avoid_multi = False
                    step_idx += 1
                # action_to_sub = np.pad(action_to_sub,pad_width=[(0, pad_len-len(action_to_sub))],  constant_values=-1)
                sub_data_byid[i][j]["action_to_sub"] = action_to_sub.astype(int).tolist()
                lens.append(len(action_to_sub))
    sub_data["episodes"] = []
    for k,v in sub_data_byid.items():
        sub_data["episodes"].extend(v)
    lens = np.array(lens)
    # print(np.sum(lens>50)/len(lens))
    print(len(sub_data["episodes"]))
    print(sub_data["episodes"][0])
    print(sub_path)
    with gzip.open(sub_path, "w") as f:
        f.write(json.dumps(sub_data).encode("utf-8"))
    return lens

In [4]:
def generate_subpath_keynodes(split):
    data_folder =  pathlib.Path("..","data","datasets","FGR2R")
    data_path = data_folder /  pathlib.Path("FGR2R_{}.json".format(split))
    with open(data_path, "r") as f:
        data = json.loads(f.read())

    sub_folder = pathlib.Path("..","data","datasets","R2R_VLNCE_FGSub")
    sub_path = sub_folder /  pathlib.Path("{}".format(split),"{}_sub.json.gz".format(split))
    with gzip.open(sub_path, "r") as f:
        sub_data = json.loads(f.read())
    if split!="test":
        gt_path = sub_folder /  pathlib.Path("{}".format(split),"{}_gt.json.gz".format(split))
        with gzip.open(gt_path, "r") as f:
            sub_gt = json.loads(f.read())

    graph_path = pathlib.Path("..","data","connectivity_graphs.pkl")
    with open(graph_path, "rb") as f:
        conn_graphs = pickle.load(f)

    # 使用trajectory_id重新组织episodes
    # 每个trajectory_id对应的第4条路经删去，并且去掉超长度限制215的路径
    sub_data_byid = {}
    trajectory_ids = set([v["trajectory_id"] for v in sub_data["episodes"]])
    for i in trajectory_ids:
        eps = []
        for v in sub_data["episodes"]:
            if v["trajectory_id"] == i and len(eps)<3:
                eps.append(v)
        sub_data_byid[i] = eps
    lens = []
    for i in tqdm(trajectory_ids):
        origin = None
        for v in data:
            if v["path_id"]==i:
                origin = v
        # 替换sub_instructions
        origin_sub = [[" ".join(v) for v in a] for a in ast.literal_eval(origin["new_instructions"])]
        origin_tokens = [[clip.tokenize(v, truncate=True).squeeze(0).tolist() for v in a] for a in origin_sub]
        sub_num = 11
        zero_sub = [0]*77
        for j in range(3):
            sub_data_byid[i][j]["sub_instruction"] = origin_sub[j]
            tmp = origin_tokens[j]
            if len(tmp)>sub_num:
                tmp = tmp[0:sub_num]
            tmp.extend([zero_sub]*(sub_num-len(tmp)))
            sub_data_byid[i][j]["sub_instruction_tokens"] = tmp
        # 计算action_to_sub
        if split!="test":
            origin_chunk = origin["chunk_view"]
            origin_path = origin["path"]
            for j in range(3):
                # 取出episode的信息
                ep = sub_data_byid[i][j]
                scene_id = ep["scene_id"].split("/")[1]
                graph = conn_graphs[scene_id]
                
                chunk = origin_chunk[j]
                keynodes = []
                for sub_idx, c in enumerate(chunk):
                    if c[0]==c[1] or c[0]==c[1]-1:
                        s = c[0]-1
                        e = c[1]-1
                        sp = graph.nodes[origin_path[s]]["position"].tolist()
                        ep = graph.nodes[origin_path[e]]["position"].tolist()
                        keynodes.append((sub_idx, (sp,ep)))
                    else:
                        s = c[0]-1
                        e = c[0]
                        while e!=c[1]:
                            sp = graph.nodes[origin_path[s]]["position"].tolist()
                            ep = graph.nodes[origin_path[e]]["position"].tolist()
                            keynodes.append((sub_idx, (sp,ep)))
                            s += 1
                            e += 1
                sub_data_byid[i][j]["keynodes"] = keynodes
    sub_data["episodes"] = []
    for k,v in sub_data_byid.items():
        sub_data["episodes"].extend(v)
    # print(np.sum(lens>50)/len(lens))
    print(len(sub_data["episodes"]))
    print(sub_data["episodes"][0])
    print(sub_path)
    with gzip.open(sub_path, "w") as f:
        f.write(json.dumps(sub_data).encode("utf-8"))

In [7]:
for split in ["train","val_seen","val_unseen","test"]:
    lens = generate_subpath(split)

  0%|          | 0/3603 [00:00<?, ?it/s]

10809
{'episode_id': 1, 'trajectory_id': 4, 'scene_id': 'mp3d/7y3sRwLe3Va/7y3sRwLe3Va.glb', 'start_position': [-16.267200469970703, 0.1518409252166748, 0.7207760214805603], 'start_rotation': [0, 0.0007963267107332633, 0, 0.9999996829318346], 'info': {'geodesic_distance': 6.425291538238525}, 'goals': [{'position': [-12.337400436401367, 0.1518409252166748, 4.213699817657471], 'radius': 3.0}], 'instruction': {'instruction_text': 'Go around the right side of the center unit and stop by the right side doorway with the dining table and mirror in it. ', 'instruction_tokens': [982, 141, 2202, 1819, 1961, 1472, 2202, 417, 2324, 103, 2104, 372, 2202, 1819, 1961, 709, 2457, 2202, 660, 2159, 103, 1390, 1138, 1173, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'reference_path': [[-16.267200469970703, 0.151840925216674

  0%|          | 0/259 [00:00<?, ?it/s]

777
{'episode_id': 185, 'trajectory_id': 2049, 'scene_id': 'mp3d/5LpN3gDmAk7/5LpN3gDmAk7.glb', 'start_position': [3.0204501152038574, -1.0843499898910522, 15.04889965057373], 'start_rotation': [0, 0.9105493663023047, 0, -0.413400352595969], 'info': {'geodesic_distance': 11.836960792541504}, 'goals': [{'position': [9.175270080566406, -1.0843499898910522, 6.389739990234375], 'radius': 3.0}], 'instruction': {'instruction_text': 'Exit the bathroom and turn left, go past the stairs and turn right then go past the kitchen table and turn left. Wait by the white couch that is facing the painting. ', 'instruction_tokens': [816, 2202, 222, 103, 2300, 1251, 982, 1589, 2202, 2058, 103, 2300, 1819, 2207, 982, 1589, 2202, 1205, 2159, 103, 2300, 1251, 9, 2379, 372, 2202, 2435, 549, 2201, 1168, 833, 2202, 1553, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

  0%|          | 0/613 [00:00<?, ?it/s]

1839
{'episode_id': 1012, 'trajectory_id': 4099, 'scene_id': 'mp3d/TbHJrupSAjP/TbHJrupSAjP.glb', 'start_position': [-1.9241100549697876, 3.4244003295898438, -7.408279895782471], 'start_rotation': [0, 0.7212540129736931, 0, -0.6926706640022687], 'info': {'geodesic_distance': 7.445135116577148}, 'goals': [{'position': [0.9583550095558167, 3.11130428314209, -4.9011101722717285], 'radius': 3.0}], 'instruction': {'instruction_text': 'walk away from sink, turn left, walk towards foot of bed, exit through doorway on left, turn right and step down two stairs and stop. ', 'instruction_tokens': [2384, 167, 936, 1972, 2300, 1251, 2384, 2260, 903, 1472, 243, 816, 2228, 709, 1480, 1251, 2300, 1819, 103, 2088, 717, 2313, 2058, 103, 2104, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'reference_path': [[-1.9241100549697876

  0%|          | 0/1136 [00:00<?, ?it/s]

3408
{'episode_id': 1681, 'trajectory_id': 0, 'scene_id': 'mp3d/yqstnuAEVhm/yqstnuAEVhm.glb', 'start_position': [6.034969806671143, 3.1383068561553955, -11.127400398254395], 'start_rotation': [0, -0.47537390589207923, 0, -0.8797838652742551], 'instruction': {'instruction_text': 'Walk into the living room from the hallway. Enter the hallway to the left. Wait inside the bedroom through the first door on the left. ', 'instruction_tokens': [2384, 1165, 2202, 1290, 1842, 936, 2202, 1036, 9, 791, 2202, 1036, 2246, 2202, 1251, 9, 2379, 1150, 2202, 246, 2228, 2202, 870, 701, 1480, 2202, 1251, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'sub_instruction': ['walk into the living room from the hallway', 'Enter the hallway to the left', 'wait inside the bedroom through the first door on the left'], 'sub_instruction_tokens':

In [43]:
for split in ["train","val_seen","val_unseen","test"]:
    lens = generate_subpath(split)
    

  0%|          | 0/3602 [00:00<?, ?it/s]

10806
{'episode_id': 1, 'trajectory_id': 4, 'scene_id': 'mp3d/7y3sRwLe3Va/7y3sRwLe3Va.glb', 'start_position': [-16.267200469970703, 0.1518409252166748, 0.7207760214805603], 'start_rotation': [0, 0.0007963267107332633, 0, 0.9999996829318346], 'info': {'geodesic_distance': 6.425291538238525}, 'goals': [{'position': [-12.337400436401367, 0.1518409252166748, 4.213699817657471], 'radius': 3.0}], 'instruction': {'instruction_text': 'Go around the right side of the center unit and stop by the right side doorway with the dining table and mirror in it. ', 'instruction_tokens': [982, 141, 2202, 1819, 1961, 1472, 2202, 417, 2324, 103, 2104, 372, 2202, 1819, 1961, 709, 2457, 2202, 660, 2159, 103, 1390, 1138, 1173, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'reference_path': [[-16.267200469970703, 0.151840925216674

  0%|          | 0/259 [00:00<?, ?it/s]

777
{'episode_id': 185, 'trajectory_id': 2049, 'scene_id': 'mp3d/5LpN3gDmAk7/5LpN3gDmAk7.glb', 'start_position': [3.0204501152038574, -1.0843499898910522, 15.04889965057373], 'start_rotation': [0, 0.9105493663023047, 0, -0.413400352595969], 'info': {'geodesic_distance': 11.836960792541504}, 'goals': [{'position': [9.175270080566406, -1.0843499898910522, 6.389739990234375], 'radius': 3.0}], 'instruction': {'instruction_text': 'Exit the bathroom and turn left, go past the stairs and turn right then go past the kitchen table and turn left. Wait by the white couch that is facing the painting. ', 'instruction_tokens': [816, 2202, 222, 103, 2300, 1251, 982, 1589, 2202, 2058, 103, 2300, 1819, 2207, 982, 1589, 2202, 1205, 2159, 103, 2300, 1251, 9, 2379, 372, 2202, 2435, 549, 2201, 1168, 833, 2202, 1553, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

  0%|          | 0/613 [00:00<?, ?it/s]

1839
{'episode_id': 1012, 'trajectory_id': 4099, 'scene_id': 'mp3d/TbHJrupSAjP/TbHJrupSAjP.glb', 'start_position': [-1.9241100549697876, 3.4244003295898438, -7.408279895782471], 'start_rotation': [0, 0.7212540129736931, 0, -0.6926706640022687], 'info': {'geodesic_distance': 7.445135116577148}, 'goals': [{'position': [0.9583550095558167, 3.11130428314209, -4.9011101722717285], 'radius': 3.0}], 'instruction': {'instruction_text': 'walk away from sink, turn left, walk towards foot of bed, exit through doorway on left, turn right and step down two stairs and stop. ', 'instruction_tokens': [2384, 167, 936, 1972, 2300, 1251, 2384, 2260, 903, 1472, 243, 816, 2228, 709, 1480, 1251, 2300, 1819, 103, 2088, 717, 2313, 2058, 103, 2104, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'reference_path': [[-1.9241100549697876

  0%|          | 0/1136 [00:00<?, ?it/s]

3408
{'episode_id': 1681, 'trajectory_id': 0, 'scene_id': 'mp3d/yqstnuAEVhm/yqstnuAEVhm.glb', 'start_position': [6.034969806671143, 3.1383068561553955, -11.127400398254395], 'start_rotation': [0, -0.47537390589207923, 0, -0.8797838652742551], 'instruction': {'instruction_text': 'Walk into the living room from the hallway. Enter the hallway to the left. Wait inside the bedroom through the first door on the left. ', 'instruction_tokens': [2384, 1165, 2202, 1290, 1842, 936, 2202, 1036, 9, 791, 2202, 1036, 2246, 2202, 1251, 9, 2379, 1150, 2202, 246, 2228, 2202, 870, 701, 1480, 2202, 1251, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 'sub_instruction': ['walk into the living room from the hallway', 'Enter the hallway to the left', 'wait inside the bedroom through the first door on the left'], 'sub_instruction_tokens':

In [23]:
print(np.sum(lens>300)/len(lens))

0.0002775464890369137


In [39]:
for split in ["train","val_seen","val_unseen"]:
    sub_folder = pathlib.Path("..","data","datasets","R2R_VLNCE_FGSub")
    gt_path = sub_folder /  pathlib.Path("{}".format(split),"{}_gt.json.gz".format(split))
    with gzip.open(gt_path, "r") as f:
        sub_gt = json.loads(f.read())
    cnt = 0
    for k, v in sub_gt.items():
        if len(v["actions"])>200:
            print(v, split)
            print(len(v["actions"]))
            cnt+=1
    print(cnt)


{'locations': [[6.730010032653809, 0.15230600535869598, -0.6780830025672913], [6.480069160461426, 0.15230600535869598, -0.6726396083831787], [6.230128288269043, 0.15230600535869598, -0.6671962738037109], [5.98018741607666, 0.15230600535869598, -0.6617529392242432], [5.730246543884277, 0.15230600535869598, -0.6563096046447754], [5.4803056716918945, 0.15230600535869598, -0.6508662700653076], [5.230364799499512, 0.15230600535869598, -0.6454229354858398], [4.980423927307129, 0.15230600535869598, -0.6399796009063721], [4.730483055114746, 0.15230600535869598, -0.6345362663269043], [4.480542182922363, 0.15230600535869598, -0.6290929317474365], [4.2306013107299805, 0.15230600535869598, -0.6236495971679688], [3.9806604385375977, 0.15230600535869598, -0.618206262588501], [3.740645170211792, 0.15230600535869598, -0.5482590198516846], [3.5006299018859863, 0.15230600535869598, -0.47831180691719055], [3.2606146335601807, 0.15230600535869598, -0.40836459398269653], [3.020599365234375, 0.1523060053586

In [43]:
split = "train"
sub_folder = pathlib.Path("..","data","datasets","R2R_VLNCE_FGSub")
sub_path = sub_folder /  pathlib.Path("{}".format(split),"{}_sub.json.gz".format(split))
with gzip.open(sub_path, "r") as f:
    sub_data = json.loads(f.read())
print(sub_data["episodes"][0]["action_to_sub"])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [42]:
ep["start_position"]

[16.461999893188477, 0.1496591567993164, -1.9354000091552734]

In [111]:
sub_data["episodes"] = []
for k,v in sub_data_byid.items():
    sub_data["episodes"].extend(v)
print(len(sub_data["episodes"]))

777


检查是否和官方发布的数据一致
很多不一致

In [78]:
import numpy as np
from tqdm.notebook import tqdm
for a in tqdm(data):
    pid = a["path_id"]
    sub_len = [len(sub) for sub in ast.literal_eval(a["new_instructions"])]
    sub_len = np.array(sorted(sub_len))

    sub_len_new = []
    inst = []
    for ep in sub_data["episodes"]:
        if ep["trajectory_id"] == pid:
            # print(ep["instruction"])
            sub_len_new.append(len(ep["sub_instruction"]))
            inst.append(ep["sub_instruction"])
    if not sub_len_new:
        continue
    sub_len_new = np.array(sorted(sub_len_new))
    if len(sub_len)!=len(sub_len_new):
        print(pid,end=", ")
    # break

  0%|          | 0/4675 [00:00<?, ?it/s]

801, 765, 246, 823, 219, 568, 724, 178, 766, 335, 

对比并手动删除episodes中多余的,FGR2R中每条路径只有三个子指令

In [79]:
tr_list = [801, 765, 246, 823, 219, 568, 724, 178, 766, 335]
del_list = [3,3,3,3,3,3,3,3,3,3]
for pid in tr_list:
    print(pid)
    for ep in sub_data_byid[pid]:
        print(ep["instruction"]["instruction_text"])
    print()

801
Walk towards the pool and then turn left when you get next to it. Turn left and stop in front of the steps to go into the pool. 
Move towards the pool by following the blue dots. Do not move the furniture. 
walk passed sitting area towards pool, stop at edge of pool and turn left. walk three feet and stop. 
Turn around and walk by the table and chairs. Take a left and wait by the pool. 

765
Go through the doors marked with a blue dot. 
Leave the room and take a right. Walk down the hall and take the last right into a room under construction. Stop outside of the utility closet. 
Exit sitting room to hall through double doors. Turn slight right and follow hallway straight. At end of hallway turn right and enter next hallway. Stop in hallway. 
exit through double doors, go down hall and down stairs, stop at bottom of stairs. 

246
Enter the house and walk straight past the living room and staircase.  When you get to the dining room table, turn left.  Enter the room in front of you an

In [91]:
for pid in tr_list:
    print(pid)
    for ep in data:
        if ep["path_id"]==pid:
            for z in ast.literal_eval(ep["new_instructions"]):
                # print(ep)
                print(", ".join([" ".join(v) for v in z]))
            print()

801
walk towards the pool, and then turn left when you get next to it, turn left and stop in front of the step to go into the pool
move towards the pool by follow the blue dot, do not move the furniture
walk pass sitting area towards pool, stop at edge of pool and turn left, walk three foot, and stop

765
go through the door mark with a blue dot
leave the room, and take a right, walk down the hall, and take the last right into a room under construction, stop outside of the utility closet
exit sitting room to hall through double door, turn slight right, and follow hallway straight, at end of hallway turn right, and enter next hallway, stop in hallway

246
enter the house, and walk straight past the living room and staircase, when you get to the dining room table turn left, enter the room in front of you, and stop in front of the table with the vase and open book on it
turn to the left, and continue walk into the house, you will walk past the dining room table, and then take a left into 

In [76]:
for ep in data:
    if ep["path_id"] == 801:
        print(len(ep["instructions"]))

4


In [70]:
len(ast.literal_eval(ep["new_instructions"]))

3

In [17]:
for ep in sub_data["episodes"]:
    if ep["trajectory_id"]==6250:
        print(ep)

{'episode_id': 9326, 'trajectory_id': 6250, 'scene_id': 'mp3d/VLzqgDo317F/VLzqgDo317F.glb', 'start_position': [2.2407100200653076, 3.2046029567718506, 5.2129998207092285], 'start_rotation': [0, -0.8199873858364342, 0, -0.5723815921822878], 'info': {'geodesic_distance': 9.14873218536377}, 'goals': [{'position': [-0.018407700583338737, 0.6382741928100586, 0.40815499424934387], 'radius': 3.0}], 'instruction': {'instruction_text': 'Walk down one flight of stairs and stop on the landing. ', 'instruction_tokens': [2384, 717, 1483, 881, 1472, 2058, 103, 2104, 1480, 2202, 1222, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
with gzip.open("../data/datasets/R2R_VLNCE_BERT/train/train_gt.json.gz") as f:
    d = json.loads(f.read())

In [25]:
a = d['1']

In [26]:
len(a["locations"])

28

In [29]:
len(a["actions"])

46

In [32]:
len([v for v in a["actions"] if v==1])

27

In [33]:
a["locations"]

[[-16.267200469970703, 0.1518409252166748, 0.7207760214805603],
 [-16.331520080566406, 0.1518409252166748, 0.9623602628707886],
 [-16.33112144470215, 0.1518409252166748, 1.2123600244522095],
 [-16.33072280883789, 0.1518409252166748, 1.4623597860336304],
 [-16.330324172973633, 0.1518409252166748, 1.7123595476150513],
 [-16.329925537109375, 0.1518409252166748, 1.9623593091964722],
 [-16.329526901245117, 0.1518409252166748, 2.2123589515686035],
 [-16.32912826538086, 0.1518409252166748, 2.4623587131500244],
 [-16.3287296295166, 0.1518409252166748, 2.7123584747314453],
 [-16.328330993652344, 0.1518409252166748, 2.962358236312866],
 [-16.327932357788086, 0.1518409252166748, 3.212357997894287],
 [-16.327533721923828, 0.1518409252166748, 3.462357759475708],
 [-16.32713508605957, 0.1518409252166748, 3.712357521057129],
 [-16.08555030822754, 0.1518409252166748, 3.7766776084899902],
 [-15.843966484069824, 0.1518409252166748, 3.8409976959228516],
 [-15.60238265991211, 0.1518409252166748, 3.9053177